In [1]:
import numpy as np
import pandas as pd
import os

from sklearn.metrics import confusion_matrix

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten

from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

from collections import Counter

/opt/miniconda3/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


## Data loading

In [2]:
path = '~/Desktop/CareerFoundry/3.1./'
pleasant = pd.read_csv(os.path.join(path, 'Data/Original/Dataset-Answers-Weather_Prediction_Pleasant_Weather.csv'))
unscaled = pd.read_csv(os.path.join(path, 'Data/Original/Dataset-weather-prediction-dataset-processed.csv'))
unscaled.head()

,DATE,MONTH,BASEL_cloud_cover,BASEL_wind_speed,BASEL_humidity,BASEL_pressure,BASEL_global_radiation,BASEL_precipitation,BASEL_snow_depth,BASEL_sunshine,...,VALENTIA_cloud_cover,VALENTIA_humidity,VALENTIA_pressure,VALENTIA_global_radiation,VALENTIA_precipitation,VALENTIA_snow_depth,VALENTIA_sunshine,VALENTIA_temp_mean,VALENTIA_temp_min,VALENTIA_temp_max
0,19600101,1,7,2.1,0.85,1.018,0.32,0.09,0,0.7,...,5,0.88,1.0003,0.45,0.34,0,4.7,8.5,6.0,10.9
1,19600102,1,6,2.1,0.84,1.018,0.36,1.05,0,1.1,...,7,0.91,1.0007,0.25,0.84,0,0.7,8.9,5.6,12.1
2,19600103,1,8,2.1,0.90,1.018,0.18,0.30,0,0.0,...,7,0.91,1.0096,0.17,0.08,0,0.1,10.5,8.1,12.9
3,19600104,1,3,2.1,0.92,1.018,0.58,0.00,0,4.1,...,7,0.86,1.0184,0.13,0.98,0,0.0,7.4,7.3,10.6
4,19600105,1,6,2.1,0.95,1.018,0.65,0.14,0,5.4,...,3,0.80,1.0328,0.46,0.00,0,5.7,5.7,3.0,8.4


## Data Wrangle

In [3]:
# Remove weather stations not included in "pleasant weather" answers
unscaled = unscaled.drop(['GDANSK_cloud_cover', 'GDANSK_humidity', 'GDANSK_precipitation', 'GDANSK_snow_depth', 'GDANSK_temp_mean', 'GDANSK_temp_min', 'GDANSK_temp_max',
                        'ROMA_cloud_cover', 'ROMA_wind_speed', 'ROMA_humidity', 'ROMA_pressure', 'ROMA_sunshine', 'ROMA_temp_mean',
                        'TOURS_wind_speed', 'TOURS_humidity', 'TOURS_pressure', 'TOURS_global_radiation', 'TOURS_precipitation', 'TOURS_temp_mean', 'TOURS_temp_min', 'TOURS_temp_max'], axis=1)

In [4]:
# Remove 2 observation types missing multiple years
unscaled['YEAR'] = unscaled['DATE'].astype(str).str[:4].astype(int)


In [5]:
long = unscaled.melt(
    id_vars=['YEAR'],
    var_name='observation',
    value_name='value'
)


In [6]:

years_per_obs = (
    long
    .groupby(['observation', 'YEAR'])['value']
    .apply(lambda x: x.isna().all())
    .reset_index(name='all_missing')
)
missing_years = (
    years_per_obs
    .groupby('observation')['all_missing']
    .sum()
    .sort_values(ascending=True)
)
missing_years
# No stations with missing years where found

observation
BASEL_cloud_cover            0
MADRID_sunshine              0
MADRID_temp_max              0
MADRID_temp_mean             0
MADRID_temp_min              0
                            ..
DUSSELDORF_wind_speed        0
HEATHROW_cloud_cover         0
HEATHROW_global_radiation    0
HEATHROW_precipitation       0
VALENTIA_temp_min            0
Name: all_missing, Length: 149, dtype: int64

In [7]:
unscaled.shape

(22950, 150)

In [8]:
unscaled.isna().sum().sort_values(ascending=True)

DATE                         0
MADRID_precipitation         0
MADRID_sunshine              0
MADRID_temp_mean             0
MADRID_temp_min              0
                            ..
DUSSELDORF_temp_max          0
HEATHROW_cloud_cover         0
HEATHROW_humidity            0
HEATHROW_global_radiation    0
YEAR                         0
Length: 150, dtype: int64

In [9]:
pleasant.isna().sum().sort_values(ascending=True)

DATE                           0
BASEL_pleasant_weather         0
BELGRADE_pleasant_weather      0
BUDAPEST_pleasant_weather      0
DEBILT_pleasant_weather        0
DUSSELDORF_pleasant_weather    0
HEATHROW_pleasant_weather      0
KASSEL_pleasant_weather        0
LJUBLJANA_pleasant_weather     0
MAASTRICHT_pleasant_weather    0
MADRID_pleasant_weather        0
MUNCHENB_pleasant_weather      0
OSLO_pleasant_weather          0
SONNBLICK_pleasant_weather     0
STOCKHOLM_pleasant_weather     0
VALENTIA_pleasant_weather      0
dtype: int64

In [10]:
unscaled.shape

(22950, 150)

In [11]:
# checking how many features eacj station has

Counter(col.split('_')[0] for col in unscaled.columns) 


Counter({'BASEL': 11,
         'DUSSELDORF': 11,
         'OSLO': 11,
         'DEBILT': 10,
         'HEATHROW': 10,
         'LJUBLJANA': 10,
         'MAASTRICHT': 10,
         'MADRID': 10,
         'SONNBLICK': 10,
         'VALENTIA': 10,
         'BELGRADE': 9,
         'BUDAPEST': 9,
         'KASSEL': 9,
         'MUNCHENB': 9,
         'STOCKHOLM': 8,
         'DATE': 1,
         'MONTH': 1,
         'YEAR': 1})

In [12]:
print([col for col in unscaled.columns if col.split('_')[0] == 'BASEL'])
print([col for col in unscaled.columns if col.split('_')[0] == 'DUSSELDORF'])
print([col for col in unscaled.columns if col.split('_')[0] == 'BELGRADE'])
print([col for col in unscaled.columns if col.split('_')[0] == 'BUDAPEST'])
print([col for col in unscaled.columns if col.split('_')[0] == 'STOCKHOLM'])
# it looks like that there is no unique feture that missing in most of the stations

['BASEL_cloud_cover', 'BASEL_wind_speed', 'BASEL_humidity', 'BASEL_pressure', 'BASEL_global_radiation', 'BASEL_precipitation', 'BASEL_snow_depth', 'BASEL_sunshine', 'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max']
['DUSSELDORF_cloud_cover', 'DUSSELDORF_wind_speed', 'DUSSELDORF_humidity', 'DUSSELDORF_pressure', 'DUSSELDORF_global_radiation', 'DUSSELDORF_precipitation', 'DUSSELDORF_snow_depth', 'DUSSELDORF_sunshine', 'DUSSELDORF_temp_mean', 'DUSSELDORF_temp_min', 'DUSSELDORF_temp_max']
['BELGRADE_cloud_cover', 'BELGRADE_humidity', 'BELGRADE_pressure', 'BELGRADE_global_radiation', 'BELGRADE_precipitation', 'BELGRADE_sunshine', 'BELGRADE_temp_mean', 'BELGRADE_temp_min', 'BELGRADE_temp_max']
['BUDAPEST_cloud_cover', 'BUDAPEST_humidity', 'BUDAPEST_pressure', 'BUDAPEST_global_radiation', 'BUDAPEST_precipitation', 'BUDAPEST_sunshine', 'BUDAPEST_temp_mean', 'BUDAPEST_temp_min', 'BUDAPEST_temp_max']
['STOCKHOLM_cloud_cover', 'STOCKHOLM_pressure', 'STOCKHOLM_global_radiation', 'STOCKHOLM_pr

In [13]:

observation_types = ['cloud_cover', 'wind_speed', 'humidity', 'pressure',
                     'global_radiation', 'precipitation', 'snow_depth', 
                     'sunshine', 'temp_mean', 'temp_min', 'temp_max']

In [14]:
# Create a dictionary to store the count of stations for each observation type
station_counts = {}

for obs in observation_types:
    columns = [col for col in unscaled.columns if col.endswith(obs)]
    
    station_counts[obs] = len(columns)

print("Number of stations covered by each observation type:")
for obs, count in station_counts.items():
    print(f"{obs}: {count} stations")

Number of stations covered by each observation type:
cloud_cover: 14 stations
wind_speed: 9 stations
humidity: 14 stations
pressure: 14 stations
global_radiation: 15 stations
precipitation: 15 stations
snow_depth: 6 stations
sunshine: 15 stations
temp_mean: 15 stations
temp_min: 15 stations
temp_max: 15 stations


Drop wind_speed and snow_depth

In [15]:
cols_to_drop = [col for col in unscaled.columns if '_wind_speed' in col or '_snow_depth' in col]
unscaled = unscaled.drop(cols_to_drop, axis=1)
unscaled.shape

(22950, 135)

In [16]:
all_columns = unscaled.columns.to_list()
observation_types = ['cloud_cover', 'humidity', 'pressure']
stations = set(col.split("_")[0] for col in all_columns)
missing = {}
for obs in observation_types:
    columns = [col for col in unscaled.columns if col.endswith(obs)]
    station_names = set([col.replace(f'_{obs}', '') for col in columns])
    missing_stations = stations - station_names
    missing[obs] = missing_stations

for obs, missing_stations in missing.items():
    print(f"\nMissing from {obs}:")
    if missing_stations:
        for station in missing_stations:
            print(station)
    else:
        print("None")


Missing from cloud_cover:
YEAR
MONTH
DATE
KASSEL

Missing from humidity:
YEAR
MONTH
STOCKHOLM
DATE

Missing from pressure:
YEAR
MUNCHENB
DATE
MONTH


Forcing the right column order

In [17]:
unscaled.columns.get_loc('HEATHROW_temp_max')

55

In [18]:
unscaled.columns.get_loc('STOCKHOLM_cloud_cover')

117

In [19]:
unscaled.columns.get_loc('MUNCHENB_cloud_cover')

91

In [20]:
# Insert new columns into "unscaled" at specific positions.
# The data for these new columns is taken from weather stations they are close to

unscaled.insert(56,'KASSEL_cloud_cover', unscaled['DUSSELDORF_cloud_cover'])
unscaled.insert(119, 'STOCKHOLM_humidity', unscaled['OSLO_humidity'])
unscaled.insert(94,'MUNCHENB_pressure',unscaled['BASEL_pressure'])

In [21]:
unscaled.columns.tolist()


['DATE',
 'MONTH',
 'BASEL_cloud_cover',
 'BASEL_humidity',
 'BASEL_pressure',
 'BASEL_global_radiation',
 'BASEL_precipitation',
 'BASEL_sunshine',
 'BASEL_temp_mean',
 'BASEL_temp_min',
 'BASEL_temp_max',
 'BELGRADE_cloud_cover',
 'BELGRADE_humidity',
 'BELGRADE_pressure',
 'BELGRADE_global_radiation',
 'BELGRADE_precipitation',
 'BELGRADE_sunshine',
 'BELGRADE_temp_mean',
 'BELGRADE_temp_min',
 'BELGRADE_temp_max',
 'BUDAPEST_cloud_cover',
 'BUDAPEST_humidity',
 'BUDAPEST_pressure',
 'BUDAPEST_global_radiation',
 'BUDAPEST_precipitation',
 'BUDAPEST_sunshine',
 'BUDAPEST_temp_mean',
 'BUDAPEST_temp_min',
 'BUDAPEST_temp_max',
 'DEBILT_cloud_cover',
 'DEBILT_humidity',
 'DEBILT_pressure',
 'DEBILT_global_radiation',
 'DEBILT_precipitation',
 'DEBILT_sunshine',
 'DEBILT_temp_mean',
 'DEBILT_temp_min',
 'DEBILT_temp_max',
 'DUSSELDORF_cloud_cover',
 'DUSSELDORF_humidity',
 'DUSSELDORF_pressure',
 'DUSSELDORF_global_radiation',
 'DUSSELDORF_precipitation',
 'DUSSELDORF_sunshine',
 'DUSS

In [22]:
unscaled.shape


(22950, 138)

In [23]:
unscaled.to_pickle(os.path.join(path, 'Data/Clean/cleaned_for_keras_DATE.pkl'))

In [24]:
unscaled.drop(columns=["DATE", "MONTH", "YEAR"], inplace=True)

In [25]:
unscaled.shape

(22950, 135)

In [26]:
pleasant.head()

,DATE,BASEL_pleasant_weather,BELGRADE_pleasant_weather,BUDAPEST_pleasant_weather,DEBILT_pleasant_weather,DUSSELDORF_pleasant_weather,HEATHROW_pleasant_weather,KASSEL_pleasant_weather,LJUBLJANA_pleasant_weather,MAASTRICHT_pleasant_weather,MADRID_pleasant_weather,MUNCHENB_pleasant_weather,OSLO_pleasant_weather,SONNBLICK_pleasant_weather,STOCKHOLM_pleasant_weather,VALENTIA_pleasant_weather
0,19600101,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,19600102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,19600103,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,19600104,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,19600105,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [27]:
pleasant.drop(columns='DATE', inplace=True)
pleasant.shape

(22950, 15)

export

In [28]:
unscaled.to_pickle(os.path.join(path, 'Data/Clean/cleaned_for_keras.pkl'))

## Convert and Reshape Data 

In [29]:
X = np.array(unscaled)
X = X.reshape(-1,15,9)
X.shape


(22950, 15, 9)

In [30]:
y = np.array(pleasant)
y.shape

(22950, 15)

## Train/ Test split

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

In [32]:
print(X_train.shape, X_test.shape)
print(y_train.shape, y_test.shape)

(17212, 15, 9) (5738, 15, 9)
(17212, 15) (5738, 15)


## CNN 

In [33]:
model = Sequential()

model.add(
    Conv1D(
        filters=64,          # changed
        kernel_size=2,
        activation='relu',
        input_shape=(15, 9)
    )
)

model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(64, activation='relu'))

# Multi-label output: one sigmoid unit per station
model.add(Dense(15, activation='sigmoid'))

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 14, 64)         │         1,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 7, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 448)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │        28,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 30,927 (120.81 KB)

 Trainable params: 30,927 (120.81 KB)

 Non-trainable params: 0 (0.00 B)

The model architecture consists of a one-dimensional convolutional layer applied across weather stations, followed by max pooling and a fully connected hidden layer.

The final output layer contains 15 sigmoid-activated neurons, corresponding to independent binary predictions for each weather station. This configuration is appropriate for multi-label classification, where multiple stations may simultaneously experience pleasant weather on the same day.

Compile and run the model

In [34]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [35]:
history = model.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=16,
    validation_split=0.2,
    verbose=2
)


Epoch 1/30
861/861 - 2s - 2ms/step - accuracy: 0.1467 - loss: 0.2508 - val_accuracy: 0.1679 - val_loss: 0.2024
Epoch 2/30
861/861 - 1s - 882us/step - accuracy: 0.1662 - loss: 0.1911 - val_accuracy: 0.1812 - val_loss: 0.1806
Epoch 3/30
861/861 - 1s - 853us/step - accuracy: 0.1792 - loss: 0.1732 - val_accuracy: 0.1748 - val_loss: 0.1649
Epoch 4/30
861/861 - 1s - 860us/step - accuracy: 0.1904 - loss: 0.1604 - val_accuracy: 0.1769 - val_loss: 0.1500
Epoch 5/30
861/861 - 1s - 852us/step - accuracy: 0.1925 - loss: 0.1500 - val_accuracy: 0.1917 - val_loss: 0.1469
Epoch 6/30
861/861 - 1s - 809us/step - accuracy: 0.1931 - loss: 0.1406 - val_accuracy: 0.2059 - val_loss: 0.1380
Epoch 7/30
861/861 - 1s - 822us/step - accuracy: 0.2064 - loss: 0.1331 - val_accuracy: 0.1792 - val_loss: 0.1357
Epoch 8/30
861/861 - 1s - 832us/step - accuracy: 0.2132 - loss: 0.1254 - val_accuracy: 0.2027 - val_loss: 0.1256
Epoch 9/30
861/861 - 1s - 846us/step - accuracy: 0.2150 - loss: 0.1200 - val_accuracy: 0.2469 - va

In [36]:
# Threshold predictions
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Overall label accuracy
overall_accuracy = (y_pred == y_test).mean()

# Accuracy per station
station_accuracy = (y_pred == y_test).mean(axis=0)

overall_accuracy, station_accuracy


180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 545us/step


(np.float64(0.9701289647960962),
 array([0.95033113, 0.9625305 , 0.97229   , 0.95712792, 0.97385849,
        0.95904496, 0.96915301, 0.97159289, 0.96131056, 0.97420704,
        0.97682119, 0.97838968, 1.        , 0.97350993, 0.97176717]))

Confusion Matrix

In [37]:
stations = {
0: 'BASEL',
1: 'BELGRADE',
2: 'BUDAPEST',
3: 'DEBILT',
4: 'DUSSELDORF',
5: 'HEATHROW',
6: 'KASSEL',
7: 'LJUBLJANA',
8: 'MAASTRICHT',
9: 'MADRID',
10: 'MUNCHENB',
11: 'OSLO',
12: 'SONNBLICK',
13: 'STOCKHOLM',
14: 'VALENTIA'

}

In [38]:
def confusion_matrix(y_true, y_pred):
    y_true = pd.Series([stations[y] for y in np.argmax(y_true, axis=1)])
    y_pred = pd.Series([stations[y] for y in np.argmax(y_pred, axis=1)])

    return pd.crosstab(y_true, y_pred, rownames=['True'], colnames=['Pred'])

In [39]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 382us/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         163       544        51      54          99      1248      94   
BELGRADE        0       548        46       8          16        45       7   
BUDAPEST        0         6        52       3           4        15       2   
DEBILT          0         0         0      10           2         6       0   
DUSSELDORF      0         0         0       0           3         5       0   
HEATHROW        0         1         0       0           0        27       0   
KASSEL          0         0         0       0           0         0       2   
LJUBLJANA       0         1         1       0           0         0       0   
MAASTRICHT      1         0         0       0           0         0       0   
MADRID          0         2         1       0           0         2       0   
MUNCHENB 

Adjusted

In [40]:
model = Sequential()

model.add(
    Conv1D(
        filters=32, #changed
        kernel_size=2,
        activation='relu',
        input_shape=(15, 9)
    )
)

model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(64, activation='relu'))

# Multi-label output: one sigmoid unit per station
model.add(Dense(15, activation='sigmoid'))

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_1 (Conv1D)               │ (None, 14, 32)         │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_1 (MaxPooling1D)  │ (None, 7, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │        14,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,983 (62.43 KB)

 Trainable params: 15,983 (62.43 KB)

 Non-trainable params: 0 (0.00 B)

Compile and run the model

In [41]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [42]:
history = model.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=32, #changed
    validation_split=0.2,
    verbose=2
)


Epoch 1/30
431/431 - 1s - 3ms/step - accuracy: 0.1407 - loss: 0.2910 - val_accuracy: 0.1644 - val_loss: 0.2256
Epoch 2/30
431/431 - 0s - 975us/step - accuracy: 0.1679 - loss: 0.2120 - val_accuracy: 0.1641 - val_loss: 0.1982
Epoch 3/30
431/431 - 0s - 916us/step - accuracy: 0.1726 - loss: 0.1940 - val_accuracy: 0.1705 - val_loss: 0.1861
Epoch 4/30
431/431 - 0s - 941us/step - accuracy: 0.1767 - loss: 0.1841 - val_accuracy: 0.1502 - val_loss: 0.1815
Epoch 5/30
431/431 - 0s - 914us/step - accuracy: 0.1786 - loss: 0.1761 - val_accuracy: 0.1618 - val_loss: 0.1767
Epoch 6/30
431/431 - 0s - 915us/step - accuracy: 0.1795 - loss: 0.1690 - val_accuracy: 0.1734 - val_loss: 0.1663
Epoch 7/30
431/431 - 0s - 906us/step - accuracy: 0.1836 - loss: 0.1633 - val_accuracy: 0.1804 - val_loss: 0.1607
Epoch 8/30
431/431 - 0s - 890us/step - accuracy: 0.1843 - loss: 0.1580 - val_accuracy: 0.1856 - val_loss: 0.1600
Epoch 9/30
431/431 - 0s - 890us/step - accuracy: 0.1819 - loss: 0.1529 - val_accuracy: 0.1809 - va

In [43]:
# Threshold predictions
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Overall label accuracy
overall_accuracy = (y_pred == y_test).mean()

# Accuracy per station
station_accuracy = (y_pred == y_test).mean(axis=0)

overall_accuracy, station_accuracy


180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 547us/step


(np.float64(0.9554200069710701),
 array([0.92976647, 0.94806553, 0.93987452, 0.94806553, 0.9475427 ,
        0.9430115 , 0.95102823, 0.94667131, 0.94684559, 0.95625654,
        0.96497037, 0.96897874, 1.        , 0.96514465, 0.97507842]))

Confusion Matrix

In [44]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 356us/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         291       635       281      15         174        90      48   
BELGRADE        3       295       150       1          38         4       9   
BUDAPEST        0         5        55       1           3         4       1   
DEBILT          0         0         0       1          13         1       2   
DUSSELDORF      1         0         0       0           3         1       0   
HEATHROW        1         0         0       0           0         9       1   
KASSEL          0         0         1       0           0         0       1   
LJUBLJANA       0         0         1       0           0         0       0   
MAASTRICHT      1         0         1       0           2         1       0   
MADRID          0         0         2       0           0         0       0   
MUNCHENB 

Adjusted

In [45]:
model = Sequential()

model.add(
    Conv1D(
        filters=32, #changed
        kernel_size=2,
        activation='relu',
        input_shape=(15, 9)
    )
)

model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(64, activation='relu'))

# Multi-label output: one sigmoid unit per station
model.add(Dense(15, activation='sigmoid'))

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_2 (Conv1D)               │ (None, 14, 32)         │           608 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_2 (MaxPooling1D)  │ (None, 7, 32)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 224)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 64)             │        14,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 15)             │           975 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,983 (62.43 KB)

 Trainable params: 15,983 (62.43 KB)

 Non-trainable params: 0 (0.00 B)

The model architecture consists of a one-dimensional convolutional layer applied across weather stations, followed by max pooling and a fully connected hidden layer.

The final output layer contains 15 sigmoid-activated neurons, corresponding to independent binary predictions for each weather station. This configuration is appropriate for multi-label classification, where multiple stations may simultaneously experience pleasant weather on the same day.

Compile and run the model

In [46]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [47]:
history = model.fit(
    X_train,
    y_train,
    epochs=30,
    batch_size=16, # changed
    validation_split=0.2,
    verbose=2
)


Epoch 1/30
861/861 - 2s - 2ms/step - accuracy: 0.1533 - loss: 0.2618 - val_accuracy: 0.1577 - val_loss: 0.2049
Epoch 2/30
861/861 - 1s - 749us/step - accuracy: 0.1577 - loss: 0.2013 - val_accuracy: 0.1542 - val_loss: 0.1877
Epoch 3/30
861/861 - 1s - 755us/step - accuracy: 0.1634 - loss: 0.1841 - val_accuracy: 0.1563 - val_loss: 0.1747
Epoch 4/30
861/861 - 1s - 813us/step - accuracy: 0.1775 - loss: 0.1716 - val_accuracy: 0.1629 - val_loss: 0.1662
Epoch 5/30
861/861 - 1s - 841us/step - accuracy: 0.1798 - loss: 0.1616 - val_accuracy: 0.1751 - val_loss: 0.1578
Epoch 6/30
861/861 - 1s - 825us/step - accuracy: 0.1829 - loss: 0.1532 - val_accuracy: 0.1807 - val_loss: 0.1486
Epoch 7/30
861/861 - 1s - 825us/step - accuracy: 0.1827 - loss: 0.1454 - val_accuracy: 0.1940 - val_loss: 0.1453
Epoch 8/30
861/861 - 1s - 840us/step - accuracy: 0.1832 - loss: 0.1393 - val_accuracy: 0.1775 - val_loss: 0.1385
Epoch 9/30
861/861 - 1s - 799us/step - accuracy: 0.1846 - loss: 0.1324 - val_accuracy: 0.1661 - va

During training, the binary cross-entropy loss decreases steadily for both training and validation sets, indicating stable optimization and no signs of divergence.

Accuracy values remain relatively low and fluctuate across epochs. This behavior is expected in multi-label classification tasks with class imbalance, where many labels are negative. In such settings, accuracy is a coarse metric and should be interpreted with caution, while loss provides a more reliable signal of learning progress.

In [48]:
# Threshold predictions
y_pred = (model.predict(X_test) > 0.5).astype(int)

# Overall label accuracy
overall_accuracy = (y_pred == y_test).mean()

# Accuracy per station
station_accuracy = (y_pred == y_test).mean(axis=0)

overall_accuracy, station_accuracy


180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 536us/step


(np.float64(0.9633786452887185),
 array([0.93272917, 0.96357616, 0.95695364, 0.95869641, 0.96758452,
        0.95067968, 0.95921924, 0.94841408, 0.95538515, 0.96758452,
        0.97542698, 0.95817358, 1.        , 0.97821541, 0.97804113]))

overall_accuracy = 0.9678633670268386

The overall label accuracy is high, reflecting the model’s strong ability to correctly predict negative (not pleasant) cases across stations. However, due to class imbalance, this metric alone may overestimate practical performance and should be interpreted alongside station-level results.

Station-level accuracy varies across locations, reflecting differences in local climate variability and class balance. Stations with more consistent weather patterns achieve higher accuracy, while stations with more variable conditions present a greater prediction challenge.

Confusion Matrix

In [49]:
# Evaluate
print(confusion_matrix(y_test, model.predict(X_test)))

180/180 ━━━━━━━━━━━━━━━━━━━━ 0s 351us/step
Pred        BASEL  BELGRADE  BUDAPEST  DEBILT  DUSSELDORF  HEATHROW  KASSEL  \
True                                                                          
BASEL         294       365       175      33          37       184      68   
BELGRADE        3       266       122       7          14        26      24   
BUDAPEST        0         0        55       2           4         4       3   
DEBILT          0         0         0      13           1         5       4   
DUSSELDORF      1         0         0       1           1         2       0   
HEATHROW        0         0         0       0           0        19       0   
KASSEL          0         0         0       0           0         0       3   
LJUBLJANA       0         0         1       0           0         0       0   
MAASTRICHT      0         0         0       0           1         2       0   
MADRID          0         0         3       0           0         0       0   
MUNCHENB 